In [98]:
# Import libraries
import numpy as np
import pandas as pd
import string
import json
from datetime import datetime

In [99]:
# Define the period and group concerned
target_test = 'post'
target_period = 'K23'
target_group = 'A'+target_period   # 'A' for intervention, 'K' for control

# Define the input files
file_ref = '../rawdata/snippets_reference_'+target_period+'.csv'
file_user_raw = '../rawdata/snippets_luval_'+target_test+'_'+target_group+'.csv'
file_eventlogs_raw = '../rawdata/eventlogs_luval_'+target_test+'_'+target_group+'.csv'
file_anonym_all = '../rawdata/anonym_all_'+target_period+'.json'
file_output1 = 'output/snippets_output_metrics_'+target_test+'_'+target_group   # Can be csv or xlsx
file_output2 = 'output/snippets_output_stats_'+target_test+'_'+target_group   # Can be csv or xlsx

# Define the relevent texts
max_num_rel_text = 3
dict_rel_text = {'S': [148, 149, 150], 
                 'T': [151, 152, 153]}


Not all students may have participated in the pre- or post-test, and even if they did, they may not have selected any snippets. To keep the output consistent with the same rows of students, all students' anonymous IDs are imported first. Event logs are also imported to check if the students participated in the test. Two additional flags (columns) will be added to indicate whether students have participated in the test **('has_test')** and whether they have marked any snippets **('has_snippet')**.

In [100]:
# Import all anonymous ID and drop those who are not in the target
with open(file_anonym_all, 'r') as f:
    anonym_all = json.load(f)
anonym_target = [item for item in anonym_all if item.startswith(target_group)]
anonym_target

['AK23S0001',
 'AK23S0002',
 'AK23S0003',
 'AK23S0004',
 'AK23S0005',
 'AK23S0006',
 'AK23S0007',
 'AK23S0008',
 'AK23S0009',
 'AK23S0010',
 'AK23S0011',
 'AK23S0012',
 'AK23S0013',
 'AK23S0014',
 'AK23S0015',
 'AK23S0016',
 'AK23S0017',
 'AK23S0018',
 'AK23S0019',
 'AK23S0020',
 'AK23S0021',
 'AK23S0022',
 'AK23S0024',
 'AK23S0025',
 'AK23S0027',
 'AK23S0028',
 'AK23S0029',
 'AK23S0030',
 'AK23S0031',
 'AK23S0032',
 'AK23S0033',
 'AK23S0034',
 'AK23S0035',
 'AK23S0038',
 'AK23S0039',
 'AK23S0040',
 'AK23S0041',
 'AK23S0042',
 'AK23S0043',
 'AK23S0044',
 'AK23S0045',
 'AK23S0047',
 'AK23S0048',
 'AK23S0049',
 'AK23S0051',
 'AK23S0052',
 'AK23S0053',
 'AK23S0055',
 'AK23S0057',
 'AK23S0058',
 'AK23S0059',
 'AK23S0060',
 'AK23S0061',
 'AK23S0062',
 'AK23S0063',
 'AK23S0064',
 'AK23S0065',
 'AK23S0066',
 'AK23S0067',
 'AK23S0068',
 'AK23S0069',
 'AK23S0070',
 'AK23S0071',
 'AK23S0074',
 'AK23S0075',
 'AK23S0076',
 'AK23S0077',
 'AK23S0078',
 'AK23S0079',
 'AK23S0080',
 'AK23S0081',
 'AK23

In [101]:
# Import the eventlogs for comparisons later
df_eventlogs = pd.read_csv(file_eventlogs_raw, sep=';')
df_eventlogs

,anonym,instance_id,action_id,action_type,from_phase,to_phase,url,text_id,detail,created
0,AK23S0001,PELAT,NaN,NAVIGATION,NaN,/taskmanagement,NaN,NaN,/taskmanagement,2023-04-17 06:09:20.454256+00
1,AK23S0001,PELAT,NaN,NAVIGATION,/taskmanagement,/assignment,NaN,NaN,/assignment,2023-04-17 06:09:20.636627+00
2,AK23S0001,PELAT,NaN,NAVIGATION,/assignment,/search,NaN,NaN,/search,2023-04-17 06:09:48.412378+00
3,AK23S0001,PELAT,2ca958fd-5b46-4cef-8df3-4d6389411224,SEARCH_EVENT,NaN,NaN,NaN,NaN,Miten tuui syntyy,2023-04-17 06:10:00.462849+00
4,AK23S0001,PELAT,ecf2c92d-419d-43c8-8db8-759bd1f9e904,SEARCH_EVENT,NaN,NaN,NaN,NaN,Miten tuuli syntyy,2023-04-17 06:10:08.553957+00
...,...,...,...,...,...,...,...,...,...,...
21782,AK23S8287,SOSKT,dc6ec4ba-e92d-4d09-9d36-61907cc50dbd,SNIPPET,NaN,NaN,NaN,153.0,Tuuli perustuu erilaisiin ilmanpaineisiin ja n...,2023-04-04 09:42:28.970717+00
21783,AK23S8287,SOSKT,NaN,NAVIGATION,/pages/153,/synthesis,NaN,NaN,/synthesis,2023-04-04 09:42:36.061094+00
21784,AK23S8287,SOSKT,NaN,NAVIGATION,/synthesis,/pages/152,NaN,NaN,/pages/152,2023-04-04 09:43:49.594261+00
21785,AK23S8287,SOSKT,NaN,NAVIGATION,/pages/152,/snippets,NaN,NaN,/snippets,2023-04-04 09:43:55.492208+00


In [102]:
# Read the reference snippets
df_ref = pd.read_csv(file_ref, index_col=0)
df_ref

,text_id,snippet_text,words
sid,,,
0,148,auringon lämpöenergiaa tarvitaan,3
1,148,vedellä on kolme olomuotoa,4
2,148,kiinteä nestemäinen ja kaasumainen,4
3,148,lämpötila vaikuttaa veden olomuotoon,4
4,148,veden olomuodon muutos,3
5,148,aurinko lämmittää maapallon pintaa,4
6,148,lämpöenergia saa veden haihtumaan,4
7,148,muuttamaan olomuotoa nesteestä vesihöyryksi,4
8,148,eri ympäristöissä haihtumista tapahtuu eri tav...,6


In [103]:
# Read the raw user snippets
df_user_raw = pd.read_csv(file_user_raw, sep=';')
df_user_raw

,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated
0,AK23S0001,PELAT,151.0,Ilma liikkuu ympärillämme koko ajan lämpötilae...,5,5.0,Oma otsikko 5,2023-04-17 06:11:46.698+00,2023-04-17 06:33:49.78743+00
1,AK23S0001,PELAT,151.0,Ympäristöönsä nähden lämpimämpi ilma kohoaa ai...,3,5.0,Oma otsikko 5,2023-04-17 06:13:30.784+00,2023-04-17 06:33:45.056144+00
2,AK23S0001,PELAT,151.0,"Mitä korkeammalle maanpinnasta noustaan, sitä ...",2,5.0,Oma otsikko 5,2023-04-17 06:13:50.372+00,2023-04-17 06:33:43.539029+00
3,AK23S0001,PELAT,151.0,"Kun ilma kohoaa riittävän ylös, se alkaa jälle...",1,5.0,Oma otsikko 5,2023-04-17 06:14:14.701+00,2023-04-17 06:33:41.906587+00
4,AK23S0001,PELAT,151.0,"Ilman kohoaminen johtuu siitä, että lämpimämp...",0,5.0,Oma otsikko 5,2023-04-17 06:14:39.004+00,2023-04-17 06:33:35.776889+00
...,...,...,...,...,...,...,...,...,...
2167,AK23S8252,SKMTT,142.0,"No, lähipäivinä siis tuulee pohjoisesta – eten...",4,1.0,tyyli1234,2023-04-13 09:33:58.342+00,NaN
2168,AK23S8287,SOSKT,151.0,Ilma liikkuu ympärillämme koko ajan lämpötilae...,3,NaN,NaN,2023-04-04 09:39:58.286+00,2023-04-04 09:40:27.791845+00
2169,AK23S8287,SOSKT,151.0,Ilman kohoamisen seurauksena alueella on vähem...,2,NaN,NaN,2023-04-04 09:40:13.299+00,NaN
2170,AK23S8287,SOSKT,152.0,Mutta mitä merkitystä tällä kaikella on tuulen...,1,NaN,NaN,2023-04-04 09:40:51.711+00,NaN


## 2. Data preprocessing

### 2.1 Checking for 'has_test' and 'has_snippet' flags and the task
'has_test' = appears in eventlogs raw data

'has_snippet' = appears in snippets raw data

In [104]:
df_flags = pd.DataFrame()
df_flags['anonym'] = anonym_target
df_flags['has_test'] = df_flags['anonym'].isin(df_eventlogs['anonym'])
df_flags['has_snippet'] = df_flags['anonym'].isin(df_user_raw['anonym'])
df_flags['task'] = pd.merge(
    df_flags,
    df_eventlogs.groupby('anonym')['instance_id'].last().reset_index(),
    on='anonym',
    how='left'
)['instance_id'].apply(lambda x: x[-1] if pd.notna(x) else None)

# An example to check if there are someone who has taken the test but not selecting a snippet
filtered_df = df_flags[(df_flags['has_test'] == True) & (df_flags['has_snippet'] == False)]
filtered_df

,anonym,has_test,has_snippet,task
35,AK23S0040,True,False,S
67,AK23S0078,True,False,T
137,AK23S0165,True,False,S
267,AK23S6120,True,False,S
271,AK23S6155,True,False,S
273,AK23S6160,True,False,T
287,AK23S7179,True,False,T


In [105]:
# Data Preprocessing for User Snippets
# (1) These rows are removed at this moment:
#    (a) Deactivated rows, which are snippets that have been unmarked;
#    (b) NaN in text_id, which are self-inputted snippets;
#    (c) Exactly repeated snippets.
# (2) Rename the index as 'usid' (user snippets ID) afterwards

df_user = df_user_raw[df_user_raw["deactivated"].isna()].copy()
df_user = df_user[~df_user["text_id"].isna()]
df_user = df_user.drop_duplicates(
    subset=["anonym", "snippet_text"]
).reset_index(drop=True)
df_user.index.name = "usid"
df_user['text_id'] = df_user['text_id'].astype(int)
df_user


,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated
usid,,,,,,,,,
0,AK23S0002,PELAT,151,Ilma liikkuu ympärillämme koko ajan lämpötilae...,0,1.0,Oma otsikko 1,2023-04-17 06:10:26.234+00,NaN
1,AK23S0002,PELAT,151,Vaikka ilman liikkeen havainnointi on haastava...,1,1.0,Oma otsikko 1,2023-04-17 06:10:45.019+00,NaN
2,AK23S0002,PELAT,151,"Tuuli, jonka tunnemme suoraan kasvoillamme, on...",2,1.0,Oma otsikko 1,2023-04-17 06:10:54.791+00,NaN
3,AK23S0002,PELAT,151,"Ilma liikkuu kuitenkin myös pystysuunnassa, mi...",3,1.0,Oma otsikko 1,2023-04-17 06:11:02.013+00,NaN
4,AK23S0002,PELAT,151,Ympäristöönsä nähden lämpimämpi ilma kohoaa ai...,4,1.0,Oma otsikko 1,2023-04-17 06:11:17.822+00,NaN
...,...,...,...,...,...,...,...,...,...
1487,AK23S8252,SKMTT,151,"Kun seuraavan kerran kuljet ulkona, voit havai...",4,2.0,tuuli5678,2023-04-13 09:31:54.943+00,NaN
1488,AK23S8252,SKMTT,142,"No, lähipäivinä siis tuulee pohjoisesta – eten...",4,1.0,tyyli1234,2023-04-13 09:33:58.342+00,NaN
1489,AK23S8287,SOSKT,151,Ilman kohoamisen seurauksena alueella on vähem...,2,NaN,NaN,2023-04-04 09:40:13.299+00,NaN


In [106]:
# Define a function for finding the max consecutive common subsequence
def longest_consecutive_common_subsequence(a, b):
    max_length = 0
    end_index = -1
    dp = [[0 for _ in range(len(b) + 1)] for _ in range(len(a) + 1)]

    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            if a[i - 1] == b[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end_index = i - 1
            else:
                dp[i][j] = 0

    if max_length == 0:
        return []

    return a[end_index - max_length + 1 : end_index + 1]

### Evaluation for each user snippets

Two properties are defined for each user snippets: **Relevant Proportion** and **Coverage Ratio**. 

**Relevant Proportion**: This ratio measures how many words from the reference snippet are matched or found within the user snippet, indicating the accuracy or precision of the match. A minimum of three consecutive words must match for inclusion in the calculation.

**Coverage Proportion**: This ratio represents the proportion of words in the user snippet relative to the reference snippet, indicating how much of the reference content the user snippet covers.

When a user snippet matches more than one reference snippet, only the reference snippet with the longest continuous match of words will be considered.

For example:
```
Reference snippet: "There are many forests in Finland."
User snippet: "forests in Finland. The"
```
The relevant proportion is 3/4.
The coverage proportion is 3/6.

In [107]:
# A nested loop: loop over all User Snippets (usid), then all Reference Snippets (rsid)
# Two new columns named 'relevant_proportion' and 'coverage_proportion' will be created in the User Snippets DataFrame

for usid, urow in df_user.iterrows():

    # if a reference snippet is found in the user snippet, turn those words to "True"
    urow_clean = (
        urow["snippet_text"]
        .strip()
        .lower()
        .translate(str.maketrans("", "", string.punctuation))
    )

    # Initialize a list of "False" values with the same length as uwords
    uwords = urow_clean.split()
    uwords_status = [False] * len(uwords)
    max_num_common = 0

    # Find the reference snippet with the longest continuous match of words 
    for rsid, rrow in df_ref.iterrows():
        if rrow["text_id"] == urow["text_id"] and not pd.isna(urow["snippet_text"]):
            rwords = rrow["snippet_text"].split()
            common_subsequence = longest_consecutive_common_subsequence(rwords, uwords)
            num_common = len(common_subsequence)
            if num_common >= 3 and num_common > max_num_common:
                max_rsid = rsid
                max_num_common = num_common
                max_start_index = uwords.index(common_subsequence[0])
                num_refernce_snippets_words = len(rrow["snippet_text"].split())

    if (max_num_common >= 3):
        num_user_snippets_words = len(urow["snippet_text"].split())
        df_user.at[usid, "relevant_rsid"] = max_rsid
        df_user.at[usid, "relevant_proportion"] = max_num_common / num_user_snippets_words
        df_user.at[usid, "coverage_proportion"] = max_num_common / num_refernce_snippets_words

# Preview the result
df_user

,anonym,instance_id,text_id,snippet_text,ordernumber,group_id,group_title,created,deactivated,relevant_rsid,relevant_proportion,coverage_proportion
usid,,,,,,,,,,,,
0,AK23S0002,PELAT,151,Ilma liikkuu ympärillämme koko ajan lämpötilae...,0,1.0,Oma otsikko 1,2023-04-17 06:10:26.234+00,NaN,24.0,0.388889,1.0
1,AK23S0002,PELAT,151,Vaikka ilman liikkeen havainnointi on haastava...,1,1.0,Oma otsikko 1,2023-04-17 06:10:45.019+00,NaN,NaN,NaN,NaN
2,AK23S0002,PELAT,151,"Tuuli, jonka tunnemme suoraan kasvoillamme, on...",2,1.0,Oma otsikko 1,2023-04-17 06:10:54.791+00,NaN,25.0,0.300000,1.0
3,AK23S0002,PELAT,151,"Ilma liikkuu kuitenkin myös pystysuunnassa, mi...",3,1.0,Oma otsikko 1,2023-04-17 06:11:02.013+00,NaN,NaN,NaN,NaN
4,AK23S0002,PELAT,151,Ympäristöönsä nähden lämpimämpi ilma kohoaa ai...,4,1.0,Oma otsikko 1,2023-04-17 06:11:17.822+00,NaN,26.0,0.312500,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1487,AK23S8252,SKMTT,151,"Kun seuraavan kerran kuljet ulkona, voit havai...",4,2.0,tuuli5678,2023-04-13 09:31:54.943+00,NaN,35.0,0.280000,1.0
1488,AK23S8252,SKMTT,142,"No, lähipäivinä siis tuulee pohjoisesta – eten...",4,1.0,tyyli1234,2023-04-13 09:33:58.342+00,NaN,NaN,NaN,NaN
1489,AK23S8287,SOSKT,151,Ilman kohoamisen seurauksena alueella on vähem...,2,NaN,NaN,2023-04-04 09:40:13.299+00,NaN,30.0,0.692308,1.0


In [108]:
df_user_encoded = pd.get_dummies(df_user, columns=['text_id'], prefix='text_id')
df_user_encoded[50:100]

,anonym,instance_id,snippet_text,ordernumber,group_id,group_title,created,deactivated,relevant_rsid,relevant_proportion,...,text_id_141,text_id_142,text_id_143,text_id_146,text_id_148,text_id_149,text_id_150,text_id_151,text_id_152,text_id_153
usid,,,,,,,,,,,,,,,,,,,,,
50,AK23S0009,PELAT,Samaan aikaan ylemmäs ilmakehään kohonnut ilma...,3,NaN,NaN,2023-04-17 06:19:10.025+00,NaN,33.0,0.347826,...,False,False,False,False,False,False,False,True,False,False
51,AK23S0009,PELAT,"Alueelle, johon viilennyt ilma laskeutuu, synt...",2,NaN,NaN,2023-04-17 06:19:33.079+00,NaN,34.0,0.187500,...,False,False,False,False,False,False,False,True,False,False
52,AK23S0009,PELAT,"Kun seuraavan kerran kuljet ulkona, voit havai...",1,NaN,NaN,2023-04-17 06:19:45.12+00,NaN,35.0,0.280000,...,False,False,False,False,False,False,False,True,False,False
53,AK23S0009,PELAT,Voit syventää ymmärrystäsi tuulesta tutustumal...,0,NaN,NaN,2023-04-17 06:19:55.758+00,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,False,False
54,AK23S0010,PELAT,lma liikkuu ympärillämme koko ajan lämpötilaer...,7,NaN,NaN,2023-04-17 06:12:16.664+00,NaN,24.0,0.857143,...,False,False,False,False,False,False,False,True,False,False
55,AK23S0010,PELAT,Ympäristöönsä nähden lämpimämpi ilma kohoaa ai...,6,NaN,NaN,2023-04-17 06:13:00.826+00,NaN,26.0,0.714286,...,False,False,False,False,False,False,False,True,False,False
56,AK23S0010,PELAT,"Kun ilma kohoaa riittävän ylös, se alkaa jäll...",5,NaN,NaN,2023-04-17 06:13:10.791+00,NaN,28.0,0.583333,...,False,False,False,False,False,False,False,True,False,False
57,AK23S0010,PELAT,Tämä pystysuuntainen liike vaikuttaa ilmanpain...,4,NaN,NaN,2023-04-17 06:13:20.332+00,NaN,29.0,0.444444,...,False,False,False,False,False,False,False,True,False,False
58,AK23S0010,PELAT,Aurinko lämmittää maapallon pintaa lämpöenergi...,3,NaN,NaN,2023-04-17 06:14:58.303+00,NaN,37.0,1.000000,...,False,False,False,False,False,False,False,False,True,False


In [109]:
# Group by 'anonym' and aggregate data
df_stat = df_user_encoded.groupby('anonym').agg({
    'instance_id': lambda x: x.iloc[-1][-1],  # Get the last character of the last 'instance_id'
    'snippet_text': 'count',                  # Count the number of snippets per anonym
    'relevant_proportion': 'mean',            # Calculate mean of relevant_proportion, automatically excludes NaN
    'coverage_proportion': 'mean'             # Calculate mean of coverage_proportion, automatically excludes NaN
}).rename(columns={
    'instance_id': 'task',
    'snippet_text': 'valid_snippets_total',
    'relevant_proportion': 'mean_relevant_proportion',
    'coverage_proportion': 'mean_coverage_proportion'
})

# creating a placeholder for relevant texts
df_stat['relevant_snippets_total'] = 0
for i in range(1,max_num_rel_text+1):
    df_stat['sum_relevant_text_'+str(i)] = 0

# For each text_id_### column, sum the True values
text_id_cols = [col for col in df_user_encoded if col.startswith('text_id_')]
for col in text_id_cols:
    df_stat['sum_' + col] = df_user_encoded.groupby('anonym')[col].sum()

# !!! HUOM !!!
# This part is task-specific! Refer to the definition of dict_rel_text at the beginning
def get_sum_relevant_text(row, i):
    task = row['task']
    text_id = dict_rel_text[task][i]
    return row['sum_text_id_'+str(text_id)]

for i in range(max_num_rel_text):
    df_stat['sum_relevant_text_'+str(i+1)] = df_stat.apply(get_sum_relevant_text, axis=1, args=(i,))
    df_stat['relevant_snippets_total'] += df_stat['sum_relevant_text_'+str(i+1)]

# Reset index to make 'anonym' a column again if needed
df_stat.reset_index(inplace=True)

# Drop the task as it will be duplicate with df_flags
df_stat = df_stat.drop('task', axis=1)

df_stat

,anonym,valid_snippets_total,mean_relevant_proportion,mean_coverage_proportion,relevant_snippets_total,sum_relevant_text_1,sum_relevant_text_2,sum_relevant_text_3,sum_text_id_113,sum_text_id_114,...,sum_text_id_141,sum_text_id_142,sum_text_id_143,sum_text_id_146,sum_text_id_148,sum_text_id_149,sum_text_id_150,sum_text_id_151,sum_text_id_152,sum_text_id_153
0,AK23S0002,7,0.333681,0.895833,7,7,0,0,0,0,...,0,0,0,0,0,0,0,7,0,0
1,AK23S0003,7,0.300000,1.000000,7,3,4,0,0,0,...,0,0,0,0,0,0,0,3,4,0
2,AK23S0004,7,0.364286,1.000000,3,3,0,0,0,0,...,4,0,0,0,0,0,0,3,0,0
3,AK23S0005,10,0.477773,1.000000,10,3,2,5,0,0,...,0,0,0,0,0,0,0,3,2,5
4,AK23S0006,6,0.196875,1.000000,6,1,5,0,0,0,...,0,0,0,0,0,0,0,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,AK23S6152,10,0.737374,1.000000,10,0,0,10,0,0,...,0,0,0,0,0,0,10,0,0,0
228,AK23S6197,2,0.273352,1.000000,2,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
229,AK23S8072,7,0.503030,1.000000,7,3,1,3,0,0,...,0,0,0,0,3,1,3,0,0,0
230,AK23S8252,5,0.280000,1.000000,1,1,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0


In [110]:
# Export the results to Excel
table1 = df_user[['anonym','snippet_text','relevant_rsid','relevant_proportion','coverage_proportion']]
table1.to_excel(file_output1+'.xlsx')

table2 = pd.merge(df_flags, df_stat, on='anonym', how='left')
table2.loc[table2['has_snippet'], 'valid_snippets_total'] = table2.loc[table2['has_snippet'], 'valid_snippets_total'].fillna(0)
table2.to_excel(file_output2+'.xlsx')